In [ ]:
# Paper Title: Developing a Sentence Level Fairness Metric Using Word Embeddings
# Coded by Ahmed Izzidien

# Install the following when running for first time 

import sys
!{sys.executable} -m pip install seaborn 
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install pyinflect
!{sys.executable} -m pip install lemminflect
!{sys.executable} -m pip install graphviz
!{sys.executable} -m pip install pydotplus
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install tensorflow #(or install using anaconda if denied access: https://docs.anaconda.com/anaconda/user-guide/tasks/tensorflow/ )
!{sys.executable} -m pip install -U pytorch-pretrained-bert
!pip install https://github.com/MartinoMensio/spacy-universal-sentence-encoder/releases/download/v0.4.3/en_use_md-0.4.3.tar.gz 
!pip install fitbert
!pip install wonderwords
!pip install pattern
!pip install spacy-universal-sentence-encoder
!{sys.executable} -m pip install plotly==5.3.1
!pip install spacy
!pip U install sentence-transformers
!pip install transformers
!pip install flair -U
!pip install -U textblob
!python -m textblob.download_corpora

In [ ]:
import lemminflect 
import gzip
import transformers
transformers.__version__
from transformers import pipeline
import pyinflect
import wonderwords
import spacy
import matplotlib.pyplot as plt
from spacy import displacy
import scipy.stats as stats
import numpy as np
import os
import pandas as pd
from pylab import rcParams
import docx
import re
import seaborn as sns
import nltk 
nltk.download('vader_lexicon')
from nltk.corpus import wordnet 
from IPython.display import Image  
import pydotplus
import matplotlib.pyplot as plt
import plotly.express as px
import re
import sympy
import torch
torch.cuda.is_available()  #(if not available: it may be that there is no GPU in the computer)
import seaborn as sns
from fitbert import FitBert
fb = FitBert()
import random
from wonderwords import RandomWord
from wonderwords import RandomSentence
s = RandomSentence()
r = RandomWord()
unmasker = pipeline('fill-mask', model='albert-xxlarge-v2')
#Load the Universal Sentence Encoder's TF Hub module
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)
nlp = spacy.load('en_core_web_sm')
from transformers import pipeline
#Load the Universal Sentence Encoder's TF Hub module
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
from flair.models import TextClassifier
from flair.data import Sentence
from tqdm import tqdm

In [ ]:
#import the data that will be tested with the code e.g. test sentences 

from fair_unfair import fair_unfair
from thirty_six_fair_unfair import thirty_six_fair_unfair 

pd.options.display.max_rows = 400

In [ ]:
#Build Sentence FairVec (S-FairVec) using abstract concepts from the psych. litrature to represent the social ontology of fairness 
 
res = "is was very responsible"
resA = [res]
resV = embed(resA)

irres = "is was very irresponsible"
irresA = [irres]
irresV = embed(irresA)

joy = "it was joyous"
joyA = [joy]
joyV = embed(joyA)

pain = "it was sad"
painA = [pain]
painV = embed(painA)

ben = "it was beneficial to society"
benA = [ben]
benV = embed(benA)

har = "it was not beneficial to society"
harA = [har]
harV = embed(harA)

#lib = "liberty"
lib = "was free to and rewarded"
libA = [lib]
libV = embed(libA)

pri = "was sent to prison and punished"
priA = [pri]
priV = embed(priA)

app = "it was beneficial"
appA = [app]
appV = embed(appA)

inapp = "it was harmful"
inappA = [inapp]
inappV = embed(inappA)


ResponsibilityV = resV - irresV 
EmotionV = joyV - painV 
ConsequenceV = libV - priV   
BeneficialV = benV - harV 
HarmV = appV - inappV  

#This is S-FairVec which combines the above vectors into one (it needs error minimisation)
SFairVec = resV - irresV + joyV - painV  + libV - priV   + benV - harV  + appV - inappV 

In [ ]:
#Function to compare S-FairVec agaist a list of the randomly selected 36 sentences

fair_s = "it was fair"
fair_sA = [fair_s]
fair_sV = embed(fair_sA)

unfair_s = "it was unfair"
unfair_sA = [unfair_s]
unfair_sV = embed(unfair_sA)

word_fair_unfair= fair_sV-unfair_sV

def run_word_fairVec(sentences_to_test):
    word_FairVecResults=[]
    lenght = len(sentences_to_test)
    sentence_embeddings = embed(sentences_to_test)
    for i in range(lenght):
        dot =  np.inner(word_fair_unfair,sentence_embeddings[i])
        word_FairVecResults.append(
            {
                'Score': dot,
                'Sentence':sentences_to_test[i],
            }

        )

    word_df_fairness=pd.DataFrame(word_FairVecResults)
    word_df_fairness['Score'] = word_df_fairness['Score'].astype(float)
    word_df_fairness['Sentence'] = word_df_fairness['Sentence'].astype(str)
    return word_df_fairness 



#Carry out word_FairVec
word_fair_results=run_word_fairVec(thirty_six_fair_unfair)


ax_word_fair = word_fair_results.plot.bar(x='Sentence', y='Score', legend=False, figsize=(14,14))
ax_word_fair.tick_params(labelbottom=True,labeltop=False)
plt.xticks(rotation = 90)
ax_word_fair.axvline(17.5, color='k', linestyle='--')
ax_word_fair.set_facecolor("white")
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.grid(color='0.3', linestyle='-', linewidth=0.1)
plt.ylabel('Cosine Similarity', fontsize=16)
plt.xlabel('Sentences', fontsize=16)
plt.ylim(-0.1, 0.1)
plt.grid(True, which='both')



plt.show()


In [ ]:
#Function to compare S-FairVec agaist the full list 200 of sentences

fair_s = "it was fair"
fair_sA = [fair_s]
fair_sV = embed(fair_sA)

unfair_s = "it was not fair"
unfair_sA = [unfair_s]
unfair_sV = embed(unfair_sA)

word_fair_unfair= fair_sV-unfair_sV

def run_word_fairVec(sentences_to_test):
    word_FairVecResults=[]
    lenght = len(sentences_to_test)
    sentence_embeddings = embed(sentences_to_test)
   
    for i in range(lenght):
        dot =  np.inner(word_fair_unfair,sentence_embeddings[i])
        word_FairVecResults.append(
            {
                'Score': dot,
                'Sentence':sentences_to_test[i],
            }

        )

    word_df_fairness=pd.DataFrame(word_FairVecResults)
    word_df_fairness['Score'] = word_df_fairness['Score'].astype(float)
    word_df_fairness['Sentence'] = word_df_fairness['Sentence'].astype(str)
    return word_df_fairness

#Carry out word_FairVec
word_fair_results=run_word_fairVec(fair_unfair)
ax_word_fair = word_fair_results.plot.bar(x='Sentence', y='Score', rot=90,figsize=(14,14), fontsize=(4), legend=False )
ax_word_fair.axvline(100.5, color='k', linestyle='--')
plt.ylabel('Cosine Similarity', fontsize=12)
plt.xlabel('Sentences', fontsize=10)
plt.xticks([])
ax_word_fair.set_facecolor("white")
plt.yticks(fontsize=10)

In [ ]:
#Function to compare S-FairVec agaist the full list 200 of sentences

fair_s = "it was fair"
fair_sA = [fair_s]
fair_sV = embed(fair_sA)

unfair_s = "it was unfair"
unfair_sA = [unfair_s]
unfair_sV = embed(unfair_sA)

word_fair_unfair= fair_sV-unfair_sV

def run_word_fairVec(sentences_to_test):
    word_FairVecResults=[]
    lenght = len(sentences_to_test)
    sentence_embeddings = embed(sentences_to_test)
    for i in range(lenght):
        dot =  np.inner(word_fair_unfair,sentence_embeddings[i])
        word_FairVecResults.append(
            {
                'Score': dot,
                'Sentence':sentences_to_test[i],
            }

        )

    word_df_fairness=pd.DataFrame(word_FairVecResults)
    word_df_fairness['Score'] = word_df_fairness['Score'].astype(float)
    word_df_fairness['Sentence'] = word_df_fairness['Sentence'].astype(str)
    return word_df_fairness

#Carry out word_FairVec
word_fair_results=run_word_fairVec(fair_unfair)
ax_word_fair = word_fair_results.plot.bar(x='Sentence', y='Score', rot=90,figsize=(14,14), fontsize=(4), legend=False )
ax_word_fair.axvline(100.5, color='k', linestyle='--')
plt.ylabel('Cosine Similarity', fontsize=11)
plt.xlabel('Sentences', fontsize=11)
plt.xticks([])
ax_word_fair.set_facecolor("white")
plt.yticks(fontsize=10)
print(word_fair_results)

In [ ]:
#Function to compare S-FairVec agaist a list of sentences

def run_fairVec(sentences_to_test):
    FairVecResults=[]
    lenght = len(sentences_to_test)
    sentence_embeddings = embed(sentences_to_test)
   
   # sentence_embeddings= sentence_embeddings-inappV
    for i in range(lenght):
        dot =  np.inner(SFairVec,sentence_embeddings[i])
        
        FairVecResults.append(
            {
                'Score': dot,
                'Sentence':sentences_to_test[i],
            }

        )
  
    df_fairness=pd.DataFrame(FairVecResults)
    df_fairness['Score'] = df_fairness['Score'].astype(float)
    df_fairness['Sentence'] = df_fairness['Sentence'].astype(str)
    return df_fairness

In [ ]:
#Carry out S-FairVec


fair_results_36=run_fairVec(thirty_six_fair_unfair)
ax_word_fair_36 = fair_results_36.plot.bar(x='Sentence', y='Score', rot=90, legend=False, figsize=(14,14))
ax_word_fair_36.axvline(17.5, color='k', linestyle='--')

plt.xticks(fontsize=15)
plt.grid(color='0.3', linestyle='-', linewidth=0.1)
plt.ylabel('Cosine Similarity', fontsize=16)
plt.xlabel('Sentences', fontsize=16)
ax_word_fair_36.grid('on', which='major', axis='x' )
ax_word_fair_36.grid('on', which='major', axis='y' )
 
ax_word_fair_36.set_facecolor("white")
plt.yticks(fontsize=10)

 
ax_word_fair.tick_params(labelbottom=True,labeltop=False)
plt.xticks(rotation = 90)
 
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)


plt.ylim(-0.3, 0.3)
plt.grid(True, which='both')

plt.show()

In [ ]:
#Carry out sentiment score of list of 36 fair unfair list and correlate

def nltk_sentiment(sentence):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    
    nltk_sentiment = SentimentIntensityAnalyzer()
    score = nltk_sentiment.polarity_scores(sentence)
    return score
nltk_results = [nltk_sentiment(row) for row in thirty_six_fair_unfair]
results_df = pd.DataFrame(nltk_results)
text_df = pd.DataFrame(thirty_six_fair_unfair, columns = ['text'])
nltk_df = text_df.join(results_df)
nltk_df['compound'].corr(fair_results_36['Score'])
print(nltk_df)

In [ ]:
#Carry out S-FairVec on full list

fair_results_200=run_fairVec(fair_unfair)
ax_word_fair_200 = fair_results_200.plot.bar(x='Sentence', y='Score', rot=90, figsize=(14,14), fontsize=(4), legend=False )
ax_word_fair_200.axvline(100.5, color='k', linestyle='--')
plt.tick_params(axis='y', labelsize=11)
plt.ylabel('Cosine Similarity', fontsize=11)
plt.xlabel('Sentences', fontsize=11)
plt.xticks([])
ax_word_fair_200.set_facecolor("white")
plt.yticks(fontsize=11)
print(fair_results_200)

In [ ]:
#Carry out sentiment score of full fair unfair list and correlate 

nltk_results = [nltk_sentiment(row) for row in fair_unfair]
results_df = pd.DataFrame(nltk_results)
text_df = pd.DataFrame(fair_unfair, columns = ['text'])
nltk_df = text_df.join(results_df)
print(nltk_df)

nltk_df['compound'].corr(fair_results_200['Score'])

In [ ]:
nltk_df['compound'].corr(fair_results_200['Score'])

In [ ]:
#Function to compare Fair-Unfair vec agaist a list of complete sentences

fa = "it was fair"# 
faE = [fa]
faV = embed(faE)

unfa = "it was unfair" 
unfaE = [unfa]
unfaV = embed(unfaE)

fairWordsVec= faV-unfaV

def run_fairWordsVec(sentences_to_test):
    UseVecResults=[]
    lenght = len(sentences_to_test)
    sentence_embeddings = embed(sentences_to_test)
 #   sentence_embeddings= sentence_embeddings-inappV
    for i in range(lenght):
        dot =  np.inner(fairWordsVec,sentence_embeddings[i])
        UseVecResults.append(
            {
                'Score': dot,
                'Sentence':sentences_to_test[i],
            }

        )

    df_usefullness=pd.DataFrame(UseVecResults)
    df_usefullness['Score'] = df_usefullness['Score'].astype(float)
    #df_wantedness_even_if_punative['Score2'] = df_wantedness_even_if_punative['Score2'].astype(float)
    df_usefullness['Sentence'] = df_usefullness['Sentence'].astype(str)
    return df_usefullness#.sort_values(by='Score', ascending=False)

In [ ]:
fw_results=run_fairWordsVec(fair_unfair)
print(fw_results)

In [ ]:
fw_results=run_fairWordsVec(fair_unfair)
sorted_results_fw=fw_results.sort_values(by='Score', ascending=False)
print(sorted_results_fw)

In [ ]:
#Display the individual fairness vectors

In [ ]:
#ResponsibilityV
DimsTableR = []
lenght = len(thirty_six_fair_unfair)
sentence_embeddings_test_s = embed(thirty_six_fair_unfair)
        
        
for i in range(lenght):
    dot =  np.inner(ResponsibilityV,sentence_embeddings_test_s[i])
    DimsTableR.append(
            {
                'Score': dot,
                'Sentence':thirty_six_fair_unfair[i],
              
            }

    )                      
                                   
responsibility=pd.DataFrame(DimsTableR)
#df3.sort_values(by='Score', ascending=False)
        
responsibility['Score'] = responsibility['Score'].astype(float)
responsibility['Sentence'] = responsibility['Sentence'].astype(str)
responsibilityVec=responsibility

ax_res = responsibility.plot.bar(x='Sentence', y='Score', rot=90, legend=False, figsize=(14,14))

ax_res.axvline(17.5, color='k', linestyle='--')
plt.xticks([])
plt.yticks(fontsize=12)
plt.grid(color='0.3', linestyle='-', linewidth=0.1)
plt.ylabel('Cosine Similarity', fontsize=18)
plt.xlabel('Sentences', fontsize=18)
ax_res.set_facecolor("white")
ax_res.grid('on', which='major', axis='x' )
ax_res.grid('on', which='major', axis='y' )
plt.ylim(-0.25, 0.25)
plt.show()



In [ ]:
result_res=responsibility.sort_values(by='Score', ascending=False) 
print(result_res)

In [ ]:
#EmotionV
DimsTableE = []
for i in range(lenght):
    dot =  np.inner(EmotionV,sentence_embeddings_test_s[i])
    DimsTableE.append(
            {
                'Score': dot,
                'Sentence':thirty_six_fair_unfair[i],
              
            }

    )                      
                                   
df4=pd.DataFrame(DimsTableE)

        
df4['Score'] = df4['Score'].astype(float)
df4['Sentence'] = df4['Sentence'].astype(str)
emoVec=df4



ax_res = df4.plot.bar(x='Sentence', y='Score', rot=90, legend=False, figsize=(14,14))
ax_res.axvline(17.5, color='k', linestyle='--')
plt.xticks([])

plt.ylim(-0.3, 0.3)
plt.yticks(fontsize=12)
plt.grid(color='0.3', linestyle='-', linewidth=0.1)
plt.ylabel('Cosine Similarity', fontsize=18)
plt.xlabel('Sentences', fontsize=18)
ax_res.set_facecolor("white")
ax_res.grid('on', which='major', axis='x' )
ax_res.grid('on', which='major', axis='y' )
plt.ylim(-0.25, 0.25)
plt.show()

In [ ]:
result_emos=df4.sort_values(by='Score', ascending=False) 
print(result_emos)

In [ ]:
#ConsequenceV
DimsTableC = []
for i in range(lenght):
    dot =  np.inner(ConsequenceV,sentence_embeddings_test_s[i])
    DimsTableC.append(
            {
                'Score': dot,
                'Sentence':thirty_six_fair_unfair[i],
              
            }

    )                      
                                   
df5=pd.DataFrame(DimsTableC)

        
df5['Score'] = df5['Score'].astype(float)
df5['Sentence'] = df5['Sentence'].astype(str)
ConsqVec=df5


ax_con = df5.plot.bar(x='Sentence', y='Score', rot=90, legend=False, figsize=(14,14))
ax_con.axvline(17.5, color='k', linestyle='--')
ax_con.set_facecolor("white")

plt.xticks([])
plt.yticks(fontsize=12)
plt.grid(color='0.3', linestyle='-', linewidth=0.1)
plt.ylabel('Cosine Similarity', fontsize=18)
plt.xlabel('Sentences', fontsize=18)


ax_con.grid('on', which='major', axis='x' )
ax_con.grid('on', which='major', axis='y' )
plt.ylim(-0.25, 0.25)
plt.show()

In [ ]:
result_consq=df5.sort_values(by='Score', ascending=False) 
print(result_consq)

In [ ]:
#BeneficialV
DimsTableM = []
for i in range(lenght):
    dot =  np.inner(BeneficialV,sentence_embeddings_test_s[i])
    DimsTableM.append(
            {
                'Score': dot,
                'Sentence':thirty_six_fair_unfair[i],
              
            }

    )                      
                                   
df6=pd.DataFrame(DimsTableM)

        
df6['Score'] = df6['Score'].astype(float)
df6['Sentence'] = df6['Sentence'].astype(str)
BenVec=df6


ax_ben = df6.plot.bar(x='Sentence', y='Score', rot=90, legend=False, figsize=(14,14))
ax_ben.axvline(17.5, color='k', linestyle='--')
ax_ben.set_facecolor("white")

plt.xticks([])
plt.yticks(fontsize=12)
plt.grid(color='0.3', linestyle='-', linewidth=0.1)
plt.ylabel('Cosine Similarity', fontsize=18)
plt.xlabel('Sentences', fontsize=18)


ax_ben.grid('on', which='major', axis='x' )
ax_ben.grid('on', which='major', axis='y' )
plt.ylim(-0.25, 0.25)


plt.show()

In [ ]:
#HarmV
DimsTableH = []
for i in range(lenght):
    dot =  np.inner(HarmV,sentence_embeddings_test_s[i])
    DimsTableH.append(
            {
                'Score': dot,
                'Sentence':thirty_six_fair_unfair[i],
              
            }

    )                      
                                   
df7=pd.DataFrame(DimsTableH)

df7['Score'] = df7['Score'].astype(float)
df7['Sentence'] = df7['Sentence'].astype(str)
HarmVec=df7

ax_harm = df7.plot.bar(x='Sentence', y='Score', rot=90, legend=False, figsize=(14,14))
ax_harm.axvline(17.5, color='k', linestyle='--')
ax_harm.set_facecolor("white")

plt.xticks([])
plt.yticks(fontsize=12)
plt.grid(color='0.3', linestyle='-', linewidth=0.1)
plt.ylabel('Cosine Similarity', fontsize=18)
plt.xlabel('Sentences', fontsize=18)


ax_harm.grid('on', which='major', axis='x' )
ax_harm.grid('on', which='major', axis='y' )
plt.ylim(-0.25, 0.25)
plt.show()


In [ ]:
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  sns.set(font_scale=6)
  fig, ax = plt.subplots(figsize=(100,100))  
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd",
      square=True, annot=True)
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")
    
def run_and_plot(messages_):
  message_embeddings_ = embed(messages_)
  plot_similarity(messages_, message_embeddings_, 90)

In [ ]:
#Test similarity score of USE for Vector phrases 
messages = [
    
    "responsible",
   
    "irresponsible",
    
    "not responsible",
   
   
]

plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True

run_and_plot(messages)

In [ ]:
#Compile Dataset D1

lenght = len(fair_unfair)
sentence_embeddings_test_s = embed(fair_unfair)

#ResponsibilityV

DimsTableR = []

for i in range(lenght):
    dot =  np.inner(ResponsibilityV,sentence_embeddings_test_s[i])
    DimsTableR.append(
            {
                'Responsibility': dot,
              
              
            }

    )                      
                                   
responsibility=pd.DataFrame(DimsTableR)
 
        
responsibility['Responsibility'] = responsibility['Responsibility'].astype(float)
responsibilityVec=responsibility

   

#EmotionV
DimsTableE = []
for i in range(lenght):
    dot =  np.inner(EmotionV,sentence_embeddings_test_s[i])
    DimsTableE.append(
            {
                'Emotion': dot,
             
              
            }

    )                      
                                   
df4=pd.DataFrame(DimsTableE)
   
df4['Emotion'] = df4['Emotion'].astype(float)
 
emoVec=df4

#Harm
DimsTableH = []
for i in range(lenght):
    dot =  np.inner(HarmV,sentence_embeddings_test_s[i])
    DimsTableH.append(
            {
                'Harm': dot,
              
              
            }

    )                      
                                   
df7=pd.DataFrame(DimsTableH)

df7['Harm'] = df7['Harm'].astype(float)
 
HarmVec=df7

#ConsequenceV
DimsTableC = []
for i in range(lenght):
    dot =  np.inner(ConsequenceV,sentence_embeddings_test_s[i])
    DimsTableC.append(
            {
                'Consequence': dot,
              
              
            }

    )                      
                                   
df5=pd.DataFrame(DimsTableC)

        
df5['Consequence'] = df5['Consequence'].astype(float)
ConsqVec=df5


#BeneficialV
DimsTableM = []
for i in range(lenght):
    dot =  np.inner(BeneficialV,sentence_embeddings_test_s[i])
    DimsTableM.append(
            {
                'Benefit': dot,
                         
            }

    )                      
                                   
df6=pd.DataFrame(DimsTableM)

        
df6['Benefit'] = df6['Benefit'].astype(float)
BenVec=df6

In [ ]:
combined = pd.concat([BenVec, ConsqVec, HarmVec, emoVec, responsibilityVec], axis=1)

In [ ]:
print(combined)

In [ ]:
#Add the labels to the dataset 

df = pd.DataFrame(index=range(0, 200), columns=['Evaluation'],
                  dtype='float')

df.iloc[0:100] = 'fair'
df.iloc[100:200] = 'unfair'
df['Evaluation'] = df['Evaluation'].astype(str)
print(df)
combined_with_correct_label = pd.concat([BenVec, ConsqVec, HarmVec, emoVec, responsibilityVec, df], axis=1)
print(combined_with_correct_label)

In [ ]:
#Perform PCA
x=combined_with_correct_label
features = ['Benefit', 'Consequence','Harm','Emotion','Responsibility']
# Separating out the features
x = combined_with_correct_label.loc[:, features].values
# Separating out the target
y = combined_with_correct_label.loc[:,['Evaluation']].values

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(combined)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf  = pd.concat([principalDf, df[['Evaluation']]], axis = 1)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('Two component PCA', fontsize = 20)
targets = ['fair', 'unfair']
colors = ['y', 'r']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['Evaluation'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
print(combined_with_correct_label)


In [ ]:
# Scatter Plot with Hue for visualizing data 
cols = ['Benefit', 'Consequence','Harm','Emotion','Responsibility', 'Evaluation']
pp = sns.pairplot(combined_with_correct_label[cols], hue='Evaluation', size=1.8, aspect=1.8, 
                  palette={"unfair": "#FF9999", "fair": "#FFE888"},
                  plot_kws=dict(edgecolor="black", linewidth=0.5))
fig = pp.fig 
fig.subplots_adjust(top=0.93, wspace=0.3)
t = fig.suptitle('Plots', fontsize=14)

plt.rcParams["axes.labelsize"] = 9

In [ ]:
#use 3 PCA components

import plotly.express as px
from sklearn.decomposition import PCA

df = combined_with_correct_label
features = ['Benefit', 'Consequence','Harm','Emotion','Responsibility']

pca = PCA()
components = pca.fit_transform(df[features])
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=df["Evaluation"]
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
# 3 PCA
import plotly.express as px
from sklearn.decomposition import PCA

df = combined_with_correct_label
X = df[['Benefit', 'Consequence','Harm','Emotion','Responsibility']]

pca = PCA(n_components=3)
components = pca.fit_transform(X)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=df['Evaluation'],
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

In [ ]:
#Perform Logistic Regression 

y=y.ravel()
from sklearn.model_selection import train_test_split
# test_size: what proportion of original data is used for test set
train_img, test_img, train_lbl, test_lbl = train_test_split( x, y, test_size=1/7.0, random_state=0)
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(train_img)
train_img = pca.transform(train_img)
test_img = pca.transform(test_img)
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(train_img, train_lbl)
logisticRegr.predict(test_img[0].reshape(1,-1))
logisticRegr.predict(test_img[0:10])
logisticRegr.score(test_img, test_lbl)
 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

#CV scores, 5 fold CV
scores = cross_val_score(logisticRegr, train_img, train_lbl, cv=5)

#Prediction and accuracy
y_pred = logisticRegr.predict(test_img)
accuracy_test = accuracy_score(test_lbl, y_pred)

#Print the summary
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print ("Test Accuracy: %0.2f" % (accuracy_test))

In [ ]:
# Score
score = logisticRegr.score(test_img, test_lbl)
print ("Accuracy: %0.2f" % (score))

#Prediction
predictions = logisticRegr.predict(test_img)
predictions

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

#confusion matrix
cm = metrics.confusion_matrix(test_lbl, predictions)
print("Confusion Matrix: \n", cm)

#Visualize the confusion matrix
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual Class');
plt.xlabel('Predicted Class');
all_sample_title = "Test Accuracy: %0.2f" % (score)
plt.title(all_sample_title, size = 15);

In [ ]:
from sklearn.metrics import recall_score, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#CV scores, 5 fold CV
scores = cross_val_score(logisticRegr, train_img, train_lbl, cv=5)

#Prediction and accuracy
y_pred = logisticRegr.predict(test_img)
accuracy_test = accuracy_score(test_lbl, y_pred)

print(classification_report(test_lbl, y_pred))#Checking performance our model with ROC Score.
#roc_auc_score(test_lbl, y_pred)

#f1=f1_score(test_lbl, y_pred)
#print(precision_score(test_lbl, y_pred))
#print(recall_score(test_lbl, y_pred))

#Print the summary
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print ("Test Accuracy: %0.2f" % (accuracy_test))

In [ ]:
#SBERT experiments
from fair_unfair import fair_unfair
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(fair_unfair)

In [ ]:
#Build Sentence FairVec (S-FairVec) using 10 abstract concepts to represent the social ontology of fairness 

Res = ["is was very responsible"]
Ires = ["is was very irresponsible"]
joy = ["it was joyous"]
pain = ["it was sad"]
ben = ["it was Benefit to society"]
har = ["it was not Benefit to society"]
lib = ["was free to and rewarded"]
pri = ["was sent to prison and punished"]
app = ["it was Benefit"]
inapp = ["it was harmful"]


EmbedRse = model.encode(Res)
EmbedIres = model.encode(Ires)
Ejoy=model.encode(joy)
Epain=model.encode(pain)
Eben=model.encode(ben)
Ehar=model.encode(har)
Elib=model.encode(lib)
Epri=model.encode(pri)
Eapp=model.encode(app)
Einapp=model.encode(inapp)


BERTResponsibilityV = EmbedRse-EmbedIres
BERTEmotionV = Ejoy - Epain 
BERTConsequenceV = Eben - Ehar   
BERTBenefitV = Elib - Epri 
BERTHarmV = Eapp - Einapp  

#This is S-FairVec which combines the above vectors into one (it needs error minimisation)
SBERTFairVec = BERTResponsibilityV+BERTEmotionV+BERTConsequenceV+BERTBenefitV+BERTHarmV

results=cosine_similarity(SBERTFairVec, sentence_embeddings[0:])
df_results=pd.DataFrame(results)
display(df_results)

In [ ]:
#Place results into a dataset for training 


res = cosine_similarity(BERTResponsibilityV, sentence_embeddings[0:])
responsibility=pd.DataFrame(res)        

emo = cosine_similarity(BERTEmotionV, sentence_embeddings[0:])
emotion = pd.DataFrame(emo)

hm = cosine_similarity(BERTHarmV, sentence_embeddings[0:])
harm = pd.DataFrame(hm)

con = cosine_similarity(BERTConsequenceV, sentence_embeddings[0:])
consequence=pd.DataFrame(con)

ben = cosine_similarity(BERTBenefitV, sentence_embeddings[0:])
Benefit = pd.DataFrame(ben)

In [ ]:
r=responsibility.T
e=emotion.T
h=harm.T
c=consequence.T
b=Benefit.T

r.reset_index(drop=True, inplace=True)
e.reset_index(drop=True, inplace=True)
h.reset_index(drop=True, inplace=True)
c.reset_index(drop=True, inplace=True)
b.reset_index(drop=True, inplace=True)

r.rename(columns={0: "Responsibility"}, inplace=True)
e.rename(columns={0: "Emotion"}, inplace=True)
h.rename(columns={0: "Harm"}, inplace=True)
c.rename(columns={0: "Consequence"}, inplace=True)
b.rename(columns={0: "Benefit"}, inplace=True)

dataset1 = pd.concat([b, c, h, e, r], axis=1)
display(dataset1)

In [ ]:
df = pd.DataFrame(index=range(0, 200), columns=['Evaluation'],
                  dtype='float')

df.iloc[0:100] = 'fair'
df.iloc[100:200] = 'unfair'
df['Evaluation'] = df['Evaluation'].astype(str)
dataset1_with_labels = pd.concat([b, c, h, e, r, df], axis=1)
print(dataset1_with_labels)

In [ ]:
features = ['Benefit', 'Consequence','Harm','Emotion','Responsibility']
# Separating out the features
x = dataset1_with_labels.loc[:, features].values
# Separating out the target
y = dataset1_with_labels.loc[:,['Evaluation']].values
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(dataset1)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf  = pd.concat([principalDf, df[['Evaluation']]], axis = 1)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('Two component PCA', fontsize = 20)
targets = ['fair', 'unfair']
colors = ['y', 'r']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['Evaluation'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# Scatter Plot with Hue for visualising data 
plt.rcParams["axes.labelsize"] = 14
sns.set(font_scale=1.5)
cols = ['Benefit', 'Consequence','Harm','Emotion','Responsibility', 'Evaluation']
pp = sns.pairplot(dataset1_with_labels[cols], hue='Evaluation', height=1.8, aspect=1.8, 
                  palette={"unfair": "#FF9999", "fair": "#FFE888"},
                  plot_kws=dict(edgecolor="black", linewidth=0.5))
fig = pp.fig 
fig.subplots_adjust(top=0.93, wspace=0.3)
t = fig.suptitle('Plots', fontsize=14)

In [ ]:
#use 3 PCA components

import plotly.express as px
from sklearn.decomposition import PCA

df = dataset1_with_labels
features = ['Benefit', 'Consequence','Harm','Emotion','Responsibility']

pca = PCA()
components = pca.fit_transform(df[features])
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=df["Evaluation"]
)
fig.update_traces(diagonal_visible=False)
fig.show()
# 3 PCA
import plotly.express as px
from sklearn.decomposition import PCA

df = dataset1_with_labels
X = df[['Benefit', 'Consequence','Harm','Emotion','Responsibility']]

pca = PCA(n_components=3)
components = pca.fit_transform(X)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=df['Evaluation'],
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()
y=y.ravel()
from sklearn.model_selection import train_test_split
# test_size: what proportion of original data is used for test set
train_img, test_img, train_lbl, test_lbl = train_test_split( x, y, test_size=1/7.0, random_state=0)
 
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(train_img)

In [ ]:
#Perform logistic regresssion
train_img = pca.transform(train_img)
test_img = pca.transform(test_img)
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(train_img, train_lbl)
logisticRegr.predict(test_img[0].reshape(1,-1))
logisticRegr.predict(test_img[0:10])
logisticRegr.score(test_img, test_lbl)

In [ ]:
from sklearn.metrics import recall_score, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#CV scores, 5 fold CV
scores = cross_val_score(logisticRegr, train_img, train_lbl, cv=5)

#Prediction and accuracy
y_pred = logisticRegr.predict(test_img)
accuracy_test = accuracy_score(test_lbl, y_pred)

print(classification_report(test_lbl, y_pred))#Checking performance our model with ROC Score.
#roc_auc_score(test_lbl, y_pred)

#f1=f1_score(test_lbl, y_pred)
#print(precision_score(test_lbl, y_pred))
#print(recall_score(test_lbl, y_pred))

#Print the summary
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print ("Test Accuracy: %0.2f" % (accuracy_test))

In [ ]:
# Score
score = logisticRegr.score(test_img, test_lbl)
print ("Accuracy: %0.2f" % (score))

#Prediction
predictions = logisticRegr.predict(test_img)
predictions

In [ ]:
#RoBERTa results 
#Uses code from Cardiff NLP MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "The jury convicted the innocent"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

roberta_list1=[]


for j in range (199):
    text = fair_unfair[j]
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        #print(f"{i+1}) {l} {np.round(float(s), 4)}")
        roberta_list1.append(
            {
                'Sentence':fair_unfair[j],
                'l score': l,
                's score': s,
            }
        )  
roberta_data=pd.DataFrame(roberta_list1)
display(roberta_data)
 

In [ ]:
print(roberta_list1)

In [ ]:
print(fair_unfair)

In [ ]:
roberta_data.to_csv("SVM_Data.csv", encoding='utf-8', index=False)
top_roberta_answer=roberta_data.iloc[::3, :]
display(top_roberta_answer)

In [ ]:
top_roberta_answer['l score'] = top_roberta_answer['l score'].str.replace('Positive','+')
top_roberta_answer['l score'] = top_roberta_answer['l score'].str.replace('Neutral','+')
top_roberta_answer['l score'] = top_roberta_answer['l score'].str.replace('Negative','-')
display(top_roberta_answer)

In [ ]:
top_roberta_answer['new score'] = top_roberta_answer['l score'] + top_roberta_answer['s score'].astype(str)

In [ ]:
top_roberta_answer.dtypes
top_roberta_answer['new score'] = pd.to_numeric(top_roberta_answer['new score'])
display(top_roberta_answer)
top_roberta_answer['new score']. round(decimals=3)

In [ ]:
#Earlier Fairness vector results from the 200 sentences read 
fairvecresults = pd.read_csv ('fairvec.csv')
print(fairvecresults)

top_roberta_answer.reset_index(drop=True)
display(top_roberta_answer)

In [ ]:
top_roberta_answer.reset_index(drop=True)
display(top_roberta_answer)


In [ ]:
top_roberta_answer.reset_index(drop=True)

In [ ]:
top_roberta_answer.reset_index(drop=True, inplace=True)
display(top_roberta_answer)

In [ ]:

#Now perform the correlation of the fariness vector results with the RoBERTa sentiment analyser results found above

fairvecresults['FairVec'].corr(top_roberta_answer['new score'])